In [ ]:
!unzip /content/Planets_Moons_Data.zip

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import Sequential
import tensorflow as tf

##Data Preparing

In [ ]:
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1.0/255.0,
    preprocessing_function=None,
    data_format=None,
    dtype=None)
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=False,
    featurewise_std_normalization=True,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=1.0/255.0,
    preprocessing_function=None,
    data_format=None,
    dtype=None)
train_generator = train_datagen.flow_from_directory("Planets_Moons_Data/Planets and Moons",target_size=(256, 256),
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    interpolation="lanczos",
                                                    subset="training")
test_generator = test_datagen.flow_from_directory("Planets_Moons_Data/Planets and Moons",target_size=(256, 256),
                                                    batch_size=128,
                                                    class_mode='categorical',
                                                    interpolation="lanczos",
                                                    subset="validation")



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, SpatialDropout2D, MaxPooling2D
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_transfer_model():
    # Load the ResNet101 model without the top (fully connected) layers
    base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

    # Freeze the pre-trained layers so they are not trainable
    for layer in base_model.layers:
        layer.trainable = False

    #custom top layers
    x = base_model.output
    x = SpatialDropout2D(0.2)(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', data_format=None)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)

    num_classes = 11
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model with your preferred optimizer and loss function
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Display the model summary
    model.summary()

    return model

In [ ]:
model= create_transfer_model()

In [ ]:
batch_size = 128
epochs = 10
model.fit(train_generator, epochs=epochs)

# Evaluate the model using the test_generator
loss, accuracy = model.evaluate(train_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Get predictions on the test data
num_test_samples = len(test_generator.filenames)
num_classes = len(test_generator.class_indices)
test_steps_per_epoch = np.ceil(num_test_samples / batch_size)
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
predicted_labels = np.argmax(predictions, axis=1)

# Create the confusion matrix
true_labels = train_generator.classes
confusion_mtx = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(confusion_mtx)

In [ ]:
# Get predictions on the test data
num_test_samples = len(train_generator.filenames)
num_classes = len(train_generator.class_indices)
test_steps_per_epoch = np.ceil(num_test_samples / batch_size)
predictions = model.predict(train_generator, steps=test_steps_per_epoch)
predicted_labels = np.argmax(predictions, axis=1)

# Create the confusion matrix
true_labels = train_generator.classes
confusion_mtx = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
confusion_mtx

In [ ]:

TN = confusion_mtx.sum() - confusion_mtx.sum(axis=0) - confusion_mtx.sum(axis=1) + np.diag(confusion_mtx)
FP = confusion_mtx.sum(axis=0) - np.diag(confusion_mtx)
TP = np.diag(confusion_mtx)
FN = confusion_mtx.sum(axis=1) - np.diag(confusion_mtx)

# Calculate percentages
total_samples = TN + FP + TP + FN
TN_percentage = (TN / total_samples) * 100.0
FP_percentage = (FP / total_samples) * 100.0
TP_percentage = (TP / total_samples) * 100.0
FN_percentage = (FN / total_samples) * 100.0

# Display TN, FP, TP, FN in percentage
print(f"True Negatives (TN) Percentage: {TN_percentage.sum():.2f}%")
print(f"False Positives (FP) Percentage: {FP_percentage.sum():.2f}%")
print(f"True Positives (TP) Percentage: {TP_percentage.sum():.2f}%")
print(f"False Negatives (FN) Percentage: {FN_percentage.sum():.2f}%")

# Plot the confusion matrix with percentages
percentage_confusion_mtx = np.array([[TN_percentage.sum(), FP_percentage.sum()],
                                    [FN_percentage.sum(), TP_percentage.sum()]])

plt.figure(figsize=(6, 6))
sns.heatmap(percentage_confusion_mtx, annot=True, cmap="Blues", fmt=".2f", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix (Percentage)')
plt.show()

In [ ]:
def our_model():
  inp = Input(shape = (256,256,3))

  x = Conv2D(128, (2,2), strides=(2,2), padding='same', activation='ReLU', use_bias=True)(inp)
  x = BatchNormalization()(x)
  x = SpatialDropout2D(0.2)(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(x)
  x = Conv2D(256, (2,2), strides=(2,2), padding='same', activation='ReLU', use_bias=True)(x)
  x = SpatialDropout2D(0.2)(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(x)
  x = Conv2D(512, (2,2), strides=(2,2), padding='same', activation='ReLU', use_bias=True)(x)
  x = SpatialDropout2D(0.2)(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(x)
  x = Flatten()(x)
  x = Dense(128, activation='ReLU')(x)
  x = Dense(11, activation='softmax')(x)

  model = Model(inputs=inp, outputs= x)
  return model

In [ ]:
def func(name_model):

    print('#####~Model => {} '.format(name_model))

    model = our_model()
    model.summary()

    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
    my_callbacks  =  [keras.callbacks.ModelCheckpoint("/content/model/model_{epoch}.h5")]
    
    history = model.fit(train_generator,
                        validation_data=test_generator,
                        epochs=48,
                        callbacks=my_callbacks,
                        verbose=1,
                        batch_size=128,)
    # Plotting Accuracy, val_accuracy, loss, val_loss
    fig, ax = plt.subplots(1, 2, figsize=(10, 3))
    ax = ax.ravel()

    for i, met in enumerate(['accuracy', 'loss']):
        ax[i].plot(history.history[met])
        ax[i].plot(history.history['val_' + met])
        ax[i].set_title('Model {}'.format(met))
        ax[i].set_xlabel('epochs')
        ax[i].set_ylabel(met)
        ax[i].legend(['Train', 'Validation'])
    plt.show()
    
    # Predict Data Test
    pred = model.predict(test_generator)
    pred = np.argmax(pred,axis=1)
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    pred = [labels[k] for k in pred]
    
    print('\033[01m              Classification_report \033[0m')
    
    print('\033[01m              Results \033[0m')
    # Results
    results = model.evaluate(test_generator, verbose=0)
    print("    Test Loss:\033[31m \033[01m {:.5f} \033[30m \033[0m".format(results[0]))
    print("Test Accuracy:\033[32m \033[01m {:.2f}% \033[30m \033[0m".format(results[1] * 100))
    
    return results

In [ ]:
def func(pre,name_model):
    print('#####~Model => {} '.format(name_model))
    pre_model = name_model(input_shape=(256,256, 3),
                   include_top=False,
                   weights='imagenet',
                   pooling='avg')
    pre_model.trainable = False
    inputs = pre_model.input
    x = Dense(64, activation='relu')(pre_model.output)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(11, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss = 'categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
    my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              mode='auto')]
    
    history = model.fit(train_generator,validation_data=test_generator,epochs=48,callbacks=my_callbacks,verbose=0)
    # Plotting Accuracy, val_accuracy, loss, val_loss
    fig, ax = plt.subplots(1, 2, figsize=(10, 3))
    ax = ax.ravel()

    for i, met in enumerate(['accuracy', 'loss']):
        ax[i].plot(history.history[met])
        ax[i].plot(history.history['val_' + met])
        ax[i].set_title('Model {}'.format(met))
        ax[i].set_xlabel('epochs')
        ax[i].set_ylabel(met)
        ax[i].legend(['Train', 'Validation'])
    plt.show()
    
    # Predict Data Test
    pred = model.predict(test_generator)
    pred = np.argmax(pred,axis=1)
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    pred = [labels[k] for k in pred]
    
    print('\033[01m              Classification_report \033[0m')
    
    print('\033[01m              Results \033[0m')
    # Results
    results = model.evaluate(test_generator, verbose=0)
    print("    Test Loss:\033[31m \033[01m {:.5f} \033[30m \033[0m".format(results[0]))
    print("Test Accuracy:\033[32m \033[01m {:.2f}% \033[30m \033[0m".format(results[1] * 100))
    
    return results